In [1]:
from gui.components import (
	ApplicationWindow,
	Container,
	TitleBar,
	DataFrame,
	Button,
	Dropdown,
    CheckBox,
    QSizePolicy
)
from gui.components.alignment import *

from gui.components.settings import theme, settings
from itertools import product

import pandas
import numpy

data = pandas.DataFrame(numpy.random.randint(0, 100, (12, 3)), columns=["Weight", "Min", "Max"])
items = ["".join(item) for item in product("abc", repeat=5)]

class Application(ApplicationWindow):
    def __init__(self):
        super().__init__(minw=800, minh=600, name=settings.app_name)
        self.window: Container = Container(shadow=True, margin=10, spacing=10, bg_color=theme.bg_one)
        self.addWidget(self.window)
        
        self.window[None] = TitleBar(self, height=50)

        self.window[None] = self.container = Container(bg_color=theme.bg_two, cstretch=[0], rstretch=[2])

        self.container[0, 0] = Button(height=50)
        self.container[1, 0, 2, 1] = weights = DataFrame[AlignTop](data.T)
        
        self.container[0, 1] = self.environment = Container[AlignTop](cstretch=0, margin=4, spacing=10)
        self.environment[0, 0] = Dropdown(width=200).addItems(items)
        self.environment[0, 1] = checkbox = CheckBox("Equipped")

        self.container[1, 1] = self.dropdowns = Container[AlignTop](margin=5, spacing=4)
        self.dropdowns[None] = Dropdown().addItems(items)
        self.dropdowns[None] = Dropdown().addItems(items)
        self.dropdowns[None] = Dropdown().addItems(items)
        
        self.container[2, 0, 1, 2] = results = DataFrame(data)
        results.setSizePolicy(QSizePolicy.Policy.MinimumExpanding, QSizePolicy.Policy.MinimumExpanding)


Application().exec()

qt.svg: Cannot open file '/home/ventus/Documents/summoners-war-optimizer/assets/icon_menu.svg', because: No such file or directory
qt.svg: Cannot open file '/home/ventus/Documents/summoners-war-optimizer/assets/icon_menu.svg', because: No such file or directory
qt.svg: Cannot open file '/home/ventus/Documents/summoners-war-optimizer/assets/icon_menu.svg', because: No such file or directory
qt.svg: Cannot open file '/home/ventus/Documents/summoners-war-optimizer/assets/icon_menu.svg', because: No such file or directory
qt.svg: Cannot open file '/home/ventus/Documents/summoners-war-optimizer/assets/icon_menu.svg', because: No such file or directory
qt.svg: Cannot open file '/home/ventus/Documents/summoners-war-optimizer/assets/icon_menu.svg', because: No such file or directory
qt.svg: Cannot open file '/home/ventus/Documents/summoners-war-optimizer/assets/icon_menu.svg', because: No such file or directory
qt.svg: Cannot open file '/home/ventus/Documents/summoners-war-optimizer/assets/ico